# This is the project in Optimization 1

In [ ]:
from functions import *

import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import default_rng
from sklearn.metrics.pairwise import pairwise_kernels


In [ ]:
np.random.seed(42)
w = np.array([1.,1.])
b = 1.

n = 100
n_A = np.random.randint(0,n)
n_B = n-n_A
margin = 5.e-1
listA, listB = TestLinear(w,b,n_A,n_B,margin)
[plt.scatter(x[0],x[1],color="r") for x in listA]
[plt.scatter(x[0],x[1],color="b") for x in listB]
plt.show()


In [ ]:
x = np.concatenate((np.array(listA),np.array(listB)))

y = np.concatenate((np.ones(n_A), -np.ones(n_B)))

G = pairwise_kernels(x, metric = kernal_linear)


print(np.shape(G))


In [ ]:

alpha0 = np.ones(n_A+n_B)
tau = 0.1
niter = 1000
C = 100000


In [ ]:
alpha = gradient_descent(alpha0, G, y, tau, niter=1000, C=C, tol=1e-8)


In [ ]:
def w_b(alpha, y, x, C=0.8):
    
    # I_s = np.where(alpha > 0 and alpha < C)
    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 and alpha[i] < C]
    
    w = np.sum(alpha[I_s]*y[I_s]*x[I_s].T, axis=1) 
    b = y[I_s[0]] - np.dot(w, x[I_s[0]])
    return w, b
    


def plot_solution(x, y, w, b):

    plt.scatter(x[:,0], x[:,1], c=y)
    plt.plot([-3, 3], [(-b - w[0] * (-3)) / w[1], (-b - w[0] * 3) / w[1]], 'k-')
    #excact solution
    plt.plot([-3, 3], [(-1 - 1 * (-3)), (-1 - 1 * 3) ], 'r--')
    plt.show()

    
w, b = w_b(alpha, y, x, C = C)

plot_solution(x, y, w, b)




In [ ]:
niter = 500
C = 0.006

In [ ]:

G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
w, b = w_b(alpha, y, x)
plot_solution(x, y, w, b)

G = pairwise_kernels(x, metric = kernal_inv_multiquadratic, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)

w, b = w_b(alpha, y, x)
plot_solution(x, y,  w, b)


G = pairwise_kernels(x, metric = kernal_laplacian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)

w, b = w_b(alpha, y, x)
plot_solution(x, y,  w, b)




In [ ]:
G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter = 100, C=0.2)


In [ ]:
w, b = w_b(alpha, y, x, C=0.02)

plot_solution(x, y,  w, b)

In [ ]:
w, b = w_b(alpha, y, x, C=0.8)

plot_solution(x, y,  w, b)

In [ ]:
def gradient_descent_linesearch(alpha0, G, y , tau0, niter, C=100, L = 10, tol = 1e-10):
    """"
    Gradient descent with backtracking line search
    
    Parameters
    ----------

    alpha0 : np.array
        Initial point
    G : np.array
        Kernel matrix
    y : np.array
        Labels
    tau0 : float
        Initial step size
    niter : int
        Number of iterations
    C : float
        Regularization parameter
    L : int
        Number of iterations before reference function is updated

    tol : float
        Tolerance for convergence

    Returns
    -------
    alpha : np.array
        Optimal alpha
    """
    alpha = alpha0
    Y = np.diag(y)
    A = np.dot(Y,np.dot(G,Y))
    tau = tau0

    f_ref = np.inf
    f_best = f(alpha, A)
    f_c = f_best
    ell = 0
    f_ks = np.zeros(niter)
    for i in range(niter):


        d_k = projection(alpha - tau*gradientf(alpha, A), y=y, Y=Y, C=C) - alpha

        if np.max(np.abs(d_k)) < tol:
            print("Converged after", i, "iterations")
            return alpha, f_ks
        
        if i%500 == 0:
            print("Iteration", i, ":", np.max(np.abs(d_k))) 
        
        f_k = f(alpha, A)
        f_ks[i] = f_k
        if f_k < f_best:
            f_best = f_k
            f_c = f_k
            ell = 0
        else:
            f_c = np.max([f_c, f_k])
            ell = ell + 1
        if ell == L:
            f_ref = f_c
            f_c = f_k
            ell = 0

        

        if f(alpha + d_k, A) > f_ref:
            dot1 = np.dot(d_k, np.dot(A, d_k))
            dot2 = np.dot(d_k, np.dot(A, alpha))
            dot3 = np.dot(alpha, np.dot(A, d_k))
            dot4 = np.sum(d_k)
            theta = - (0.5*dot2 + 0.5 *dot3 - dot4)/dot1
            # print("theta", theta, np.shape(alpha), np.shape(d_k))
            
        else:
            theta = 1

        alpha = alpha + theta * d_k
        
        tau = BB_step_length(alpha-theta*d_k, alpha, gradientf, A, taumax=1e5, taumin=1e-5)


    print("Did not converge after", niter, "iterations")
    
    return alpha, f_ks

In [ ]:
alpha0 = np.ones(n_A+n_B)*0.5

alpha_test1, fks = gradient_descent_linesearch(alpha0, G, y, tau, niter=5000, C=0.8, L=10, tol=1e-5)
alpha_test2 = gradient_descent(alpha0, G, y, tau, niter=5000, C=0.8, tol=1e-3)


In [ ]:
plt.scatter(range(len(fks[1:-1])), fks[1:-1])
plt.show()
plt.scatter(range(len(alpha0)),alpha0)
plt.scatter( range(len(alpha_test1)),alpha_test1)
plt.show()

In [ ]:
np.random.seed(0)
alpha0 = np.random.rand((n_A+n_B))
print("alpha0", alpha0)

In [ ]:
alpha0 = np.random.rand((n_A+n_B))

niter = 3000
alpha_line = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.006, L=10, tol=1e-7)
alpha_grad = gradient_descent(alpha0, G, y, tau, niter, C=0.006, tol=1e-7)



In [ ]:
alpha0 = np.ones(n_A+n_B)
niter = 3000

alpha_line1 = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.6, L=10, tol=1e-7)
alpha_grad1 = gradient_descent(alpha0, G, y, tau, niter, C=0.6, tol=1e-3)


In [ ]:
alpha0 = np.ones(n_A+n_B)* 0.5
niter = 5000

alpha_line2 = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.6, L=10, tol=1e-7)

In [ ]:
plt.scatter(range(len(alpha0)), alpha0)
plt.scatter(range(len(alpha_line[0])), alpha_line[0], label="line search")
# plt.scatter(range(len(alpha_test1)),alpha_test1, label="line search test")
plt.legend()
plt.show()
# print(alpha_line)

In [ ]:

w, b= w_b(alpha_grad, y, x)
print(w,b)
plot_solution(x, y,  w, b)

w, b= w_b(alpha_line[0], y, x)
print(w,b)
plot_solution(x, y, w, b)


w, b = w_b(alpha_line1[0], y, x)
print(w,b)
plot_solution(x, y,  w, b)



w, b= w_b(alpha_line2[0], y, x)
print(w,b)

plot_solution(x, y, w, b)



In [ ]:
plt.scatter(range(len(fks[1:-1])), fks[1:-1], label="line search test")
plt.scatter(range(len(alpha_line[1])), alpha_line[1], label="line search")
plt.legend()
plt.show()

In [ ]:

def w_SVM(alpha, x, y, point_x , kernel = kernal_gaussian):

    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 ]

    w = np.sum(alpha[I_s] * y[I_s] * kernel(x[I_s], point_x)) 
    return w

def b_SVM(alpha, x, y, kernel = kernal_gaussian, C=0.6):

    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 and alpha[i] < C]
    
    b = y[I_s[0]]- np.sum(alpha[I_s]*y[I_s]*kernel(x[I_s], x[I_s[0]]))
    return b

points = np.linspace(-3, 3, 100)
# print(alpha)
w = [w_SVM(alpha, x, y, p) for p in points]
b = b_SVM(alpha, x, y)

plt.scatter(points, w+b, label="w_SVM")
plt.scatter(x[:,0], x[:,1], c=y)
plt.show()
# print("w, b", w, b)

# plt.plot(alpha_line1[0], label="line search")

In [ ]:
def projection_AL(vector, lower_bounds, upper_bounds):
    dimension = len(vector)
    proj_vector = np.array([])
    for k in range(0, dimension):
        if vector[k]<lower_bounds[k]:
            proj_vector = np.append(proj_vector, np.array([lower_bounds[k]]))
        elif vector[k] > upper_bounds[k]:
            proj_vector = np.append(proj_vector, np.array([upper_bounds[k]]))
        else:
            proj_vector = np.append(proj_vector, np.array([vector[k]]))
    return proj_vector

In [ ]:
def BCLM(wbxis_0, d, M, C, lambd_0, mu_0, tol_1, tol_2, maxiter): #Algoritme 17.4 i boka
    
    tol_1_k = 1/mu_0
    tol_2_k = 1/mu_0**(0.1)

    wbxis_k = wbxis_0
    lambd_k = lambd_0

    mu_k = mu_0

    for _ in range(0, maxiter):

        wbxis_k = 0#solution to projected gradient descent with tolerance tol_1_k
        distance_k = 0#wbxis_k - projection

        c_k = 0#constraints evaluated at wbxis_k
        c_k_norm = np.linalg.norm(c_k)

        if c_k_norm <= tol_2_k:
            
            if c_k_norm <= tol_2 and distance_k <= tol_1:
                return wbxis_k, lambd_k
            
            lambd_k = lambd_k - mu_k*c_k
            tol_1_k = tol_1_k/mu_k
            tol_2_k = tol_2_k/mu_k**(0.9)
        
        else:
            mu_k = 100*mu_k
            tol_1_k = tol_1_k/mu_k
            tol_2_k = tol_2_k/mu_k**(0.1)

In [ ]:
def AL(w, b, xi, s, lambd, mu, x, y, C): #kontroller at dette er rett
    AL = 0.5*np.linalg.norm(w)**2
    for i in range(0, len(xi)):
        indreprod = np.inner(w, x[i])
        AL = AL + C*xi[i] - lambd[i]*(y[i]*(indreprod + b) + xi[i] - s[i] - 1) + 0.5*mu*(y[i]*(indreprod + b) + xi[i] - s[i] - 1)**2
    return AL

#print(AL(np.array([1,1]),1, np.array([1,1]),np.array([1,1]), np.array([1,1]), 2, np.array([np.array([1,1]), np.array([1,1])]), np.array([1,1]), 1))

In [ ]:
def grad_AL(w, b, xi, s, lambd, mu, x, y, C): #Kontroller at dette er rett
    
    d = len(w)
    M = len(xi)
    
    grad_AL = np.array([])
    
    #Elements from w
    for k in range(0, d):
        grad_k = w[k]
        for i in range(0, M):
            indresum = 0
            for l in range(0, d):
                indresum = indresum + 2*x[i][k]*x[i][l]*w[l]
            grad_k = grad_k - lambd[i]*y[i]*x[i][k] + 0.5*mu*(y[i]**2 * indresum + 2*y[i]**2 * b * x[i][k] + 2*y[i]*xi[i]*x[i][k] - 2*y[i]*s[i]*x[i][k] -2*y[i]*x[i][k])
        grad_AL = np.append(grad_AL, np.array([grad_k]))
    
    #Elements from b
    grad_b = 0
    for i in range(0, M):
        grad_b = grad_b - lambd[i]*y[i] + 0.5*mu*(2*y[i]**2*b + 2*y[i]**2*np.inner(w, x[i]) + 2*y[i]*xi[i] - 2*y[i]*s[i] - 2*y[i])
    grad_AL = np.append(grad_AL, np.array([grad_b]))

    #Elements from xi
    for i in range(0, M):
        grad_xi = C - lambd[i] + 0.5*mu*(2*xi[i] + 2*y[i]*np.inner(w, x[i]) + 2*y[i]*b - 2*s[i] - 2)
        grad_AL = np.append(grad_AL, np.array([grad_xi]))

    #Elements from s
    for i in range(0, M):
        grad_s = -lambd[i] + 0.5*mu*(2*s[i] - 2*y[i]*np.inner(w, x[i]) - 2*y[i]*b - 2*xi[i] + 2)
        grad_AL = np.append(grad_AL, np.array([grad_s]))
    
    return grad_AL

#print(grad_AL(np.array([1,1]),1, np.array([1,1]),np.array([1,1]), np.array([1,1]), 2, np.array([np.array([1,1]), np.array([1,1])]), np.array([1,1]), 1))